# Lab Tests Cross-Sectional Similarity Matrices
This notebook takes in all lab test cross-sectional data and computes the similarity matrices

In [73]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
import os
import pandas as pd
import numpy as np
import re, math
from sklearn.metrics.pairwise import cosine_similarity


# my functions for cross-sectional similarity
from crossSectionSimilarity import *
from simMatrix import simMatrix

In [65]:
input_dir = "../data/patientData/"
output_dir = "../data/simMatrices/"

In [66]:
files = os.listdir(input_dir)
files = [file for file in files if re.search("lab tests_cross sect", file)]
files

['lab tests_cross sect_antibodies.csv',
 'lab tests_cross sect_blood.csv',
 'lab tests_cross sect_breakdown products.csv',
 'lab tests_cross sect_enzymes.csv',
 'lab tests_cross sect_hepatitis.csv',
 'lab tests_cross sect_other lab.csv',
 'lab tests_cross sect_proteins.csv']

In [67]:
# we remove patients that do not have a lot of data from this analysis
def removeLowDataPts(df, column_perc = .5):    
    # if number of missing higher then remove
    missing_cutoff = math.floor(len(df.columns)*column_perc)
    
    # extract total count and filter patients
    df['total_missing'] = df.isnull().sum(axis=1)
    filtered_df = df[df['total_missing'] <= missing_cutoff]
    filtered_df.drop(columns = 'total_missing', inplace = True)
    
    # print stats and return filtered dataframe
    print(f"Original Number of Patients: {len(df)}")
    print(f"Filtered Number of Patients: {len(filtered_df)}")
    
    return filtered_df

In [68]:
# single run through of the pipeline to generate cosine similarity
# these functions will likely be pushed to crossSectionSimilarity 
def prepData(df, column_perc = .5):
    
    # filter out patients with low data
    filtered_df = removeLowDataPts(df, column_perc)
    
    # normalize our data (mean = 0, sd = 1)
    scaled_df = normalizeDF(filtered_df)
    
    return scaled_df

### Compute the Similarity Matrices
Run the entire process

I am aware that this could be made significantly faster, but it does not matter for now. I suspect that I will finish the base analysis and visualization early. At which point, I can improve upon certain areas.

In [81]:
import warnings
warnings.filterwarnings("ignore")

for file in files:
    print(file)
    df = pd.read_csv(input_dir + file)
    processed_df = prepData(df)
    
    # initialize our similarity matrix class and patient combos 
    lab_sim = simMatrix("lab tests_" + re.search("_.*_(.*)\\.csv$", file).group(1))
    processed_df, col_combos = columnCombos(processed_df)
    
    # now compute similarity between each patient-patient
    for i, col_tuple in enumerate(col_combos):
        cosine, cosine_melted = computeCosine(processed_df, col_tuple)
        lab_sim.insertRow(cosine_melted)
        if any(cosine_melted.duplicated(subset = ['patient_1', 'patient_2'])):
            print("breaking")
            break
            
        try:
            if i % round(len(col_combos)*.1) ==0:
                print(f"Round {i} of {len(col_combos)}")
        except:
            pass
    
    # export
    print("Exporting...")
    lab_sim.export(output_dir)
    display(lab_sim.similarities)
    print("\n")
    

lab tests_cross sect_antibodies.csv
Original Number of Patients: 250
Filtered Number of Patients: 250
Exporting...


,patient_1,patient_2,similarity
0,252,252,1.000000
1,279,252,0.279399
2,357,252,0.999969
3,433,252,0.927209
4,634,252,-0.024586
...,...,...,...
62495,96152,99197,0.548144
62496,96364,99197,0.176565
62497,97547,99197,0.999972
62498,98622,99197,0.519501




lab tests_cross sect_blood.csv
Original Number of Patients: 2880
Filtered Number of Patients: 2824
Round 0 of 990
Round 99 of 990
Round 198 of 990
Round 297 of 990
Round 396 of 990
Round 495 of 990
Round 594 of 990
Round 693 of 990
Round 792 of 990
Round 891 of 990
Exporting...


,patient_1,patient_2,similarity
0,31958,27144,-0.194081
0,31958,17470,0.614240
1,31958,20182,0.435990
2,31958,28325,-0.617907
3,31958,50782,0.745398
...,...,...,...
95,54005,95803,0.675393
96,72644,95803,0.573192
97,78557,95803,0.474538
98,81556,95803,-0.162816




lab tests_cross sect_breakdown products.csv
Original Number of Patients: 2880
Filtered Number of Patients: 2680
Round 0 of 2701
Round 270 of 2701
Round 540 of 2701
Round 810 of 2701
Round 1080 of 2701
Round 1350 of 2701
Round 1620 of 2701
Round 1890 of 2701
Round 2160 of 2701
Round 2430 of 2701
Round 2700 of 2701
Exporting...


...

,patient_1,patient_2,similarity
0,226,226,1.000000
1,252,226,0.359656
2,279,226,0.902887
3,314,226,0.154535
4,357,226,0.356283
...,...,...,...
334079,95816,99197,0.553746
334080,96152,99197,0.713750
334081,96686,99197,0.685834
334082,97547,99197,0.528597




lab tests_cross sect_other lab.csv
Original Number of Patients: 2880
Filtered Number of Patients: 2855
Round 0 of 28
Round 3 of 28
Round 6 of 28
Round 9 of 28
Round 12 of 28
Round 15 of 28
Round 18 of 28
Round 21 of 28
Round 24 of 28
Round 27 of 28
Exporting...


,patient_1,patient_2,similarity
0,422,1284,0.044325
1,1734,1284,0.106324
2,2919,1284,0.572960
3,4118,1284,0.486037
4,5060,1284,0.280722
...,...,...,...
90907,99740,98622,0.625766
90908,99768,98622,0.282522
90909,99791,98622,0.405550
90910,99830,98622,0.815628




lab tests_cross sect_proteins.csv
Original Number of Patients: 2880
Filtered Number of Patients: 1415
Round 0 of 210
Round 21 of 210
Round 42 of 210
Round 63 of 210
Round 84 of 210
Round 105 of 210
Round 126 of 210
Round 147 of 210
Round 168 of 210
Round 189 of 210
Exporting...


,patient_1,patient_2,similarity
0,117,797,0.998451
1,689,797,0.436063
2,984,797,0.444394
3,2486,797,0.514419
4,3052,797,0.055637
...,...,...,...
1576,93077,92954,0.923224
1577,93209,92954,0.998009
1578,96218,92954,0.946637
1579,97529,92954,-0.069723
